# Read and write images

In [2]:
import os
import bigfish.stack as stack

In [3]:
# hard-code the paths of our input and output directories
path_input = "../data/input"
path_output = "../data/output"

During a typical **Fluorescence In Situ Hybridization (FISH)** experiment several multichannel 2D or 3D images are acquired. This notebook shows how to organize and read these images. For some more advanced techniques permitting multiplexed RNA detection, several independent acquisitions rounds are acquired for each position. This means, that we will work with a maximum of 5 dimensions:  
- one dimension r for the hybridization **rounds**
- one dimension c for the **channesl**
- three **spatial** dimensions z,y and x

In big-fish, we described images thus as 5D tensors at the most. However, most of the analysis are applied at the channel level, i.e. directly on the 3D or 2D images.

## Experiment, fields of view and recipes

We use **experiment** to describe one experimental condition that is imaged. Typically, several **fields of view (FoV)** are then acquired per **experiment**. All images from the same experiment have to be in the same input directory. Images of each fov and channel are stored separately either as 3D or 2D images. Furhter, each iamge should have appropriate name including information about the experiment, the fov, the channel and the round.

To load these images with big-fish, **we define recipes that describe how all relevant information about the image can be inferred from its name**. 

In [4]:
os.listdir(path_input)

['.gitignore']

For example, in the input directory two 3D images correspondig to two channels (dapi and FISH) from the same fov and the same experiment are present. In order, to read these images with big-fish, we have to define the appropriate recipe:

In [4]:
recipe = {
    "fov": "fov_1",
    "c": ["dapi", "smfish"],
    "opt": "experiment_1",
    "ext": "tif",
    "pattern": "opt_c_fov.ext"}

In this recipe, we specify different **keys**:
- the **fov** (*fov_1*) shared by the two files
- an **optional term** (*experiment_1*) shared by the two files
- a list of keys to distinguish the different **channels** (*dapi* and *smfish*)
- the **extension** name (*tif*) shared by the two files
- `**general pattern** of the filenames (*opt_c_fov.ext* as a template for *experiment_1_{c}\_fov_1.tif*)

We can use the elements provided in the recipe to recognize the filenames of the different files, read them and gather them accordingly in order to build a 5D image: **(round, channel, z, y, x)**.

In [1]:
# function to check if your recipe is correctly defined and files actually exist
stack.check_recipe(recipe, data_directory=path_input)

NameError: name 'stack' is not defined

__Note 1:__ A recipe can take into account others elements. The general pattern excepted, these elements are all optional and depend of the filenames we want to recognize. If a 3D image is decomposed in 2D layers, a __z__ key would be necessary in order to correctly stack the 2D images along the z axis (in the right order). The same logic happens if we want to stack layers from different rounds to build our final image, with a **r** key.

    {
    "fov": str or List[str], (optional)
    "z": str or List[str],   (optional)
    "c": str or List[str],   (optional)
    "r": str or List[str],   (optional)
    "ext": str,              (optional)
    "opt": str,              (optional)
    "pattern": str
    }
    
__Note 2:__ If you have a large number of images to read, keeping the same template to name your files will simplify your life.

### Build a multichannel image with a recipe

In [6]:
image = stack.build_stack(recipe, input_folder=path_input) 
print("\r shape: {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

 shape: (1, 2, 23, 650, 500)
 dtype: uint16


As expected, we return a 3D image with 1 round and 2 channels. 

### Build several multichannel images with recipes

If we want to read several multichannel images, we can define a **generator** (files are read and image is build only when it is called). The first step consists in **mapping different recipes with their relative input directory**:

In [7]:
# as an example, we just duplicate the same recipe
data_map = [
    (recipe, path_input),
    (recipe, path_input),
    (recipe, path_input),
]

# function to check if your mapping (recipe, input directory) is correct
stack.check_datamap(data_map)

True

In [8]:
image_generator = stack.build_stacks(data_map)
for i, image in enumerate(image_generator):
    print("Image {0}".format(i))
    print("\r shape: {0}".format(image.shape))
    print("\r dtype: {0}".format(image.dtype))

Image 0
 shape: (1, 2, 23, 650, 500)
 dtype: uint16
Image 1
 shape: (1, 2, 23, 650, 500)
 dtype: uint16
Image 2
 shape: (1, 2, 23, 650, 500)
 dtype: uint16


__Note:__ A generator can be used to read several images from the same recipe if you provide several FoV in this recipe.

In [9]:
recipe = {
    "fov": ["fov_1", "fov_1"],
    "c": ["dapi", "smfish"],
    "opt": "experiment_1",
    "ext": "tif",
    "pattern": "opt_c_fov.ext"}
data_map = [(recipe, path_input)]
image_generator = stack.build_stacks(data_map)
for i, image in enumerate(image_generator):
    print("Image {0}".format(i))
    print("\r shape: {0}".format(image.shape))
    print("\r dtype: {0}".format(image.dtype))

Image 0
 shape: (1, 2, 23, 650, 500)
 dtype: uint16
Image 1
 shape: (1, 2, 23, 650, 500)
 dtype: uint16


### Build a multichannel image without recipe

To load an image without defining a recipe, we need to list the **paths** of the different layers to stack together, in the right order.

In [10]:
# list paths. of the files to stack
path_dapi = os.path.join(path_input, "experiment_1_dapi_fov_1.tif")
path_smfish = os.path.join(path_input, "experiment_1_smfish_fov_1.tif")
paths = [path_dapi, path_smfish]

# load the layers and build the image
image = stack.build_stack_no_recipe(paths)
print("\r shape: {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

 shape: (1, 2, 23, 650, 500)
 dtype: uint16


## Read and write arrays

If you do not need to read different layers and rearrange them in a specific order, you can directly read files from their path:
- for *.png*, *.jpg*, *.jpeg*, *.tif* or *.tiff* extensions use `bigfish.stack.read_image`
- for *.dv* extension use `bigfish.stack.read_dv`
- for *.npy* extension use `bigfish.stack.read_array`
- for *.npz* extension use `bigfish.stack.read_compressed`
- for *.csv* extension use `bigfish.stack.read_array_from_csv`

In [11]:
path_dapi = os.path.join(path_input, "experiment_1_dapi_fov_1.tif")
image = stack.read_image(path_dapi)
print("\r shape: {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

 shape: (23, 650, 500)
 dtype: uint16


Same logic if you want to save images or arrays:
- for *.png*, *.jpg*, *.jpeg*, *.tif* or *.tiff* extensions use `bigfish.stack.save_image`
- for *.npy* extension use `bigfish.stack.save_array`
- for *.csv* extension use `bigfish.stack.save_array_to_csv`